In [1]:
import pandas as pd 
import numpy as np

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 50)

In [3]:
df = pd.read_csv('jikan_final.csv')

In [4]:
df.shape

(26720, 36)

In [5]:
cf1 = pd.read_csv('anime_ratings0.csv')

In [6]:
cf2 = pd.read_csv('anime_ratings1.csv')

In [7]:
cf3 = pd.read_csv('anime_ratings2.csv')

In [8]:
cf4 = pd.read_csv('anime_ratings3.csv')

In [63]:
user = pd.concat([cf1,cf2,cf3,cf4],axis=0)

In [64]:
user.shape

(3752106, 5)

In [65]:
df.nunique()

mal_id             26564
url                26564
images             26364
trailer             4920
approved               1
titles             26564
title              26563
title_english      10998
title_japanese     25462
title_synonyms     12463
type                   9
source                17
episodes             250
status                 3
airing                 2
aired              16116
duration             333
rating                 6
score                559
scored_by           8712
rank               16055
popularity         20364
members            11508
favorites           1901
synopsis           21510
background          2556
season                 4
year                  65
broadcast            623
producers           4701
licensors            265
studios             1681
genres               962
explicit_genres        1
themes               948
demographics           8
dtype: int64

In [66]:
user.nunique()

User ID        35141
Username       35140
Anime ID       14037
Anime Title    14067
Score             10
dtype: int64

In [13]:
df.drop_duplicates(inplace=True)

In [67]:
user.drop_duplicates(inplace=True)

In [15]:
counts = user['User ID'].value_counts()
filtered_user = user[user["User ID"].isin(counts[counts>100].index)]

In [74]:
counts1 = user['Anime ID'].value_counts()
filtered_user = user[user["Anime ID"].isin(counts1[counts1>10].index)]

In [75]:
filtered_user.shape

(3732628, 5)

In [18]:
filtered_user.nunique()

User ID        35138
Username       35137
Anime ID        8782
Anime Title     8813
Score             10
dtype: int64

In [19]:
df1 = df[df['mal_id'].isin(filtered_user['Anime ID'])]

In [20]:
df1.shape

(8782, 36)

In [21]:
df1 = df1[df1.favorites != 0]
df1 =df1[df1.members>=3000]


In [22]:
df1.shape

(7896, 36)

In [23]:
df1.isna().sum()

mal_id                0
url                   0
images                0
trailer               0
approved              0
titles                0
title                 0
title_english      2495
title_japanese        5
title_synonyms        0
type                  0
source                0
episodes             19
status                0
airing                0
aired                 0
duration              0
rating                1
score                 0
scored_by             0
rank                738
popularity            0
members               0
favorites             0
synopsis             28
background         6239
season             4272
year               4272
broadcast             0
producers             0
licensors             0
studios               0
genres                0
explicit_genres       0
themes                0
demographics          0
dtype: int64

In [24]:
df1.dropna(subset=['synopsis','rating'],inplace=True)

In [25]:
import ast 
df1.producers = df1.producers.apply(ast.literal_eval)
df1.images = df1.images.apply(ast.literal_eval)
df1.trailer = df1.trailer.apply(ast.literal_eval)
df1.titles = df1.titles.apply(ast.literal_eval)
df1.aired = df1.aired.apply(ast.literal_eval)
df1.broadcast = df1.broadcast.apply(ast.literal_eval)
df1.licensors = df1.licensors.apply(ast.literal_eval)
df1.studios = df1.studios.apply(ast.literal_eval)
df1.genres = df1.genres.apply(ast.literal_eval)
df1.themes = df1.themes.apply(ast.literal_eval)
df1.demographics = df1.demographics.apply(ast.literal_eval)


In [26]:
def extract_info(row):
    # Extract producer names
    producer_names = [producer['name'] for producer in row['producers']]
    licensors_names = [licensor['name'] for licensor in row['licensors']]
    studios_names = [studio['name'] for studio in row['studios']]
    genres = [genre['name'] for genre in row['genres']]
    themes = [theme['name'] for theme in row['themes']]
    demographics = [dg['name'] for dg in row['demographics']]
    
    # Extract embed_url from trailer
    embed_url = row['trailer']['embed_url'] if row['trailer'] else None
    aired = row['aired']['string'] if row['aired'] else None
    # Extract large_image_url from images
    large_image_url = row['images']['jpg']['large_image_url'] if row['images'] else None
    
    return pd.Series([producer_names, licensors_names,studios_names,genres,themes,demographics,embed_url,aired, large_image_url])

# Apply the function to each row of the DataFrame
df1[['producers','licensors','studios','genres','themes','demographics','trailer','aired','image']] = df1.apply(extract_info, axis=1)


In [27]:
df1 = df1[~df1['studios'].apply(lambda x: x == [])]
df1 = df1[~df1['genres'].apply(lambda x: x == [])]

In [28]:
df1.shape

(7550, 37)

In [29]:
df1 = df1.reset_index(drop=True)

In [30]:
import re

pattern = r"\[Written by MAL Rewrite\]|\(.*Source:.*\)" 

# Removing the pattern using regular expressions
df1['synopsis'] = df1['synopsis'].str.replace(pattern, '', regex=True).values


In [31]:
def remove_newline_numbers(text):
    text = text.replace('\n', ' ')
    text = re.sub(r'\d+', ' ', text)
    text = re.sub(r'[^\w\s]', ' ', text)
    return text.lower()

In [32]:
df1['synopsis_cleaned'] = df1.synopsis.apply(remove_newline_numbers)

In [33]:
import spacy
nlp = spacy.load('en_core_web_sm')
df1['synopsis_cleaned'] = df1['synopsis_cleaned'].apply(lambda x: " ".join([token.lemma_ for token in nlp(x) if not token.is_stop]))

In [34]:
rating_map = {
    "PG-13 - Teens 13 or older": "PG-13",
    "R - 17+ (violence & profanity)": "R17",
    "Rx - Hentai": "Rx",
    "R+ - Mild Nudity": "R+",
    "G - All Ages": "G",
    "PG - Children": "PG"
}

# Use the map to replace the values in the 'rating' column
df1['rating'] = df1['rating'].replace(rating_map)

In [35]:
df1['themes'] = df1['themes'].apply(lambda x:["unknown_theme"] if x == [] else x )
df1['demographics'] = df1['demographics'].apply(lambda x:["unknown_demographics"] if x == [] else x )

In [36]:
def get_season(x):
    spring = ["Mar","Apr","May"]
    summer = ["Jun","Jul","Aug"]
    fall = ["Sep","Oct","Nov"]
    winter = ["Dec","Jan","Feb"]
    y = x[:3]
    if y in spring:
        return "spring"
    elif y in winter:
        return "winter"
    elif y in fall:
        return "fall"
    elif y in summer:
        return "summer"

In [37]:
df1.season = df1.aired.apply(get_season)

In [38]:
df1.year = df1.aired.str.split(',').str[1].str[1:5]

In [39]:
def fill_na(row):
    if pd.isna(row['year']):
        if len(row['aired']) == 4:
            return row['aired']
        elif len(row['aired']) == 12:
            return row['aired'][:4]
        else:
            return row['aired'][4:8]
    else:
        return row['year']

# Apply the function to each row of the DataFrame
df1['year'] = df1.apply(fill_na, axis=1)

In [40]:
df1.season = df1.season.fillna("unknownseason")

In [41]:
data = df1[['mal_id', 'url', 'trailer', 'title',
       'title_english', 'type', 'source',
       'episodes', 'status', 'aired', 'duration', 'rating', 'score',
       'scored_by', 'rank', 'popularity', 'members', 'favorites', 'synopsis','synopsis_cleaned',
       'background', 'season', 'year', 'producers', 'licensors',
       'studios', 'genres', 'themes', 'demographics',
       'image']] 

In [42]:
def one_hot_encode(df, column):
    # Convert list of genres to string
    df[column] = df[column].apply(lambda x: ','.join(x))

    # Apply one-hot encoding and return the result
    return df[column].str.get_dummies(sep=',')

In [43]:
genres_df = one_hot_encode(data, 'genres')
producers_df = one_hot_encode(data, 'producers')
studios_df = one_hot_encode(data, 'studios')
themes_df = one_hot_encode(data, 'themes')
demographics_df = one_hot_encode(data, 'demographics')
licensors_df = one_hot_encode(data, 'licensors')

C:\Users\91877\AppData\Local\Temp\ipykernel_26712\758311016.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].apply(lambda x: ','.join(x))
C:\Users\91877\AppData\Local\Temp\ipykernel_26712\758311016.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].apply(lambda x: ','.join(x))
C:\Users\91877\AppData\Local\Temp\ipykernel_26712\758311016.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index

In [44]:
status_df = data.status.str.get_dummies()
season_df = data.season.str.get_dummies()
type_df = data.type.str.get_dummies()
source_df = data.source.str.get_dummies()
rating_df = data.rating.str.get_dummies()

In [45]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [46]:
vectorizer = TfidfVectorizer()  # Adjust max_features as needed
overview_matrix = vectorizer.fit_transform(data['synopsis_cleaned'])

In [47]:
overview_matrix = overview_matrix.toarray()
overview_df = pd.DataFrame(overview_matrix)

In [48]:
from sklearn.decomposition import PCA
num_components = 1000

# Apply PCA for dimensionality reduction
pca = PCA(n_components=num_components)
pca_data = pca.fit_transform(overview_df)

In [49]:
pca_data = pd.DataFrame(pca_data)

In [50]:
combined_features = pd.concat([pca_data,type_df,demographics_df,genres_df,themes_df],axis=1)

In [51]:
combined_features.shape

(7550, 1087)

In [52]:
from sklearn.metrics.pairwise import cosine_similarity
similarity_matrix = cosine_similarity(combined_features)

In [53]:
def recommend(anime):
    index = data[(data['title'] == anime) | (data['title_english'] == anime)].index[0]
    distances = sorted(list(enumerate(similarity_matrix[index])),reverse=True,key= lambda x:x[1])
    for i in distances[1:10]:
        
        print(data.iloc[i[0]].title,"---",i[1])

In [54]:
similarity_matrix.shape

(7550, 7550)

In [568]:
recommend("One Piece")

Dragon Quest: Dai no Daibouken (TV) --- 0.9365238089585742
Magi: Sinbad no Bouken (TV) --- 0.9336259451140854
Nanatsu no Taizai: Mokushiroku no Yonkishi --- 0.9330165727001656
Magi: The Labyrinth of Magic --- 0.9301092023083048
Magi: The Kingdom of Magic --- 0.924763955521359
Dragon Quest: Yuusha Abel Densetsu --- 0.9247098696331919
Fairy Tail (2014) --- 0.9246122119234116
Bleach: Sennen Kessen-hen --- 0.9209406056301614
Bleach: Sennen Kessen-hen - Ketsubetsu-tan --- 0.9199147441000843


In [588]:
recommend("Kuroko no Basket 2nd Season")

Kuroko no Basket 3rd Season --- 0.9641560194266621
Kuroko no Basket --- 0.9634486933100973
Slam Dunk --- 0.9308214541475688
Diamond no Ace --- 0.9226699201029991
Ahiru no Sora --- 0.9199789190672889
Diamond no Ace: Second Season --- 0.9187406350934829
Haikyuu!! To the Top Part 2 --- 0.9152527331645482
Haikyuu!! Second Season --- 0.9123803642692943
Diamond no Ace: Act II --- 0.910899041470866


In [57]:
import pickle
pickle.dump(data,open('anime.pkl','wb'))
pickle.dump(similarity_matrix,open('similarity.pkl','wb'))

In [633]:
filtered_user[filtered_user['Anime Title'].str.contains("Bleach")]

,user_id,anime_id,Anime Title,Score,user_id_encoded,anime_id_encoded,Score_scaled
62,26721,1686,Bleach Movie 1: Memories of Nobody,9,5054,1437,0.888889
63,26721,4835,Bleach Movie 3: Fade to Black - Kimi no Na wo ...,9,5054,2813,0.888889
64,26721,762,Bleach: Memories in the Rain,8,5054,676,0.777778
65,26721,834,Bleach: The Sealed Sword Frenzy,9,5054,729,0.888889
390,26730,1686,Bleach Movie 1: Memories of Nobody,9,5056,1437,0.888889
...,...,...,...,...,...,...,...
3026861,48024,41467,Bleach: Sennen Kessen-hen,8,9612,6790,0.777778
3026862,48024,53998,Bleach: Sennen Kessen-hen - Ketsubetsu-tan,8,9612,7440,0.777778
3026863,48024,834,Bleach: The Sealed Sword Frenzy,5,9612,729,0.444444
3027259,48052,269,Bleach,6,9614,243,0.555556


In [76]:
filtered_user = filtered_user[filtered_user['Anime ID'].isin(data.mal_id)]

In [77]:
filtered_user.nunique()

User ID        35122
Username       35121
Anime ID        7550
Anime Title     7579
Score             10
dtype: int64

In [83]:
counts = filtered_user['User ID'].value_counts()
filtered_user = filtered_user[filtered_user["User ID"].isin(counts[counts>100].index)]

In [84]:
filtered_user.nunique()

User ID        11858
Username       11857
Anime ID        7550
Anime Title     7576
Score             10
dtype: int64

In [86]:
filtered_user = filtered_user.iloc[:, [0,2,3,4]]

In [88]:
filtered_user.rename(columns={'User ID':'user_id','Anime ID':'anime_id'},inplace=True)

C:\Users\91877\AppData\Local\Temp\ipykernel_26712\2816907588.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_user.rename(columns={'User ID':'user_id','Anime ID':'anime_id'},inplace=True)


In [89]:
user_ids = pd.Categorical(filtered_user["user_id"])
filtered_user["user_id_encoded"] = user_ids.codes

anime_ids = pd.Categorical(filtered_user["anime_id"])
filtered_user["anime_id_encoded"] = anime_ids.codes

In [90]:
filtered_user = filtered_user.reset_index(drop=True)

In [91]:
from sklearn.preprocessing import MinMaxScaler

In [92]:
minmax = MinMaxScaler()
filtered_user["Score_scaled"] = minmax.fit_transform(filtered_user[["Score"]])

In [93]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    filtered_user[["user_id_encoded", "anime_id_encoded"]], filtered_user["Score_scaled"], test_size=0.2, random_state=40 , shuffle=True
)


In [94]:
import tensorflow as tf
from tensorflow import keras

In [116]:
num_users = len(set(X_train["user_id_encoded"]))  # Assuming unique user IDs
num_animes = len(set(X_train["anime_id_encoded"]))  # Assuming unique anime IDs
embedding_dim = 32  # Adjust dimensionality as needed

In [117]:
user_input = keras.layers.Input(name='user_encoded',shape=(1,))
anime_input = keras.layers.Input(name='anime_encoded',shape=(1,))

In [118]:
user_embeddings = keras.layers.Embedding(num_users, embedding_dim, name='user_embedding')(user_input)
anime_embeddings = keras.layers.Embedding(num_animes, embedding_dim,name='anime_embedding')(anime_input)

In [119]:
dot_product = keras.layers.Dot(name='dot_product', normalize=True,axes=2)([user_embeddings, anime_embeddings])
flattened = keras.layers.Flatten()(dot_product)

In [120]:
dense = keras.layers.Dense(64, activation='relu')(flattened)

In [121]:
output = keras.layers.Dense(1, activation="sigmoid")(dense)  # Optional bias can be added before this layer

In [122]:
model = keras.Model(
    inputs=[user_input, anime_input], outputs=output
)

In [123]:
model.compile(
    optimizer="adam", loss="mse", metrics=["mse", "mae"]  # Add more metrics as needed
)

In [124]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 user_encoded (InputLayer)   [(None, 1)]                  0         []                            
                                                                                                  
 anime_encoded (InputLayer)  [(None, 1)]                  0         []                            
                                                                                                  
 user_embedding (Embedding)  (None, 1, 32)                379456    ['user_encoded[0][0]']        
                                                                                                  
 anime_embedding (Embedding  (None, 1, 32)                241600    ['anime_encoded[0][0]']       
 )                                                                                          

In [125]:
model.fit(
    [X_train['user_id_encoded'], X_train['anime_id_encoded']],  # Separate user and anime IDs
    y_train,
    epochs=10,  # Adjust as needed
    batch_size=64,  # Adjust as needed
    validation_data=([X_val['user_id_encoded'], X_val['anime_id_encoded']], y_val),
)


Epoch 1/10
37843/37843 [==============================] - 541s 14ms/step - loss: 0.0307 - mse: 0.0307 - mae: 0.1363 - val_loss: 0.0246 - val_mse: 0.0246 - val_mae: 0.1196
Epoch 2/10
37843/37843 [==============================] - 546s 14ms/step - loss: 0.0216 - mse: 0.0216 - mae: 0.1113 - val_loss: 0.0206 - val_mse: 0.0206 - val_mae: 0.1090
Epoch 3/10
37843/37843 [==============================] - 550s 15ms/step - loss: 0.0184 - mse: 0.0184 - mae: 0.1022 - val_loss: 0.0196 - val_mse: 0.0196 - val_mae: 0.1062
Epoch 4/10
37843/37843 [==============================] - 571s 15ms/step - loss: 0.0168 - mse: 0.0168 - mae: 0.0977 - val_loss: 0.0192 - val_mse: 0.0192 - val_mae: 0.1054
Epoch 5/10
37843/37843 [==============================] - 581s 15ms/step - loss: 0.0159 - mse: 0.0159 - mae: 0.0948 - val_loss: 0.0191 - val_mse: 0.0191 - val_mae: 0.1047
Epoch 6/10
37843/37843 [==============================] - 608s 16ms/step - loss: 0.0153 - mse: 0.0153 - mae: 0.0928 - val_loss: 0.0190 - val_mse:

In [126]:
# Assume you have test data
X_test_user = X_val['user_id_encoded']
X_test_item = X_val['anime_id_encoded']

# Make predictions
predictions = model.predict([X_test_user, X_test_item])


18922/18922 [==============================] - 51s 3ms/step


In [433]:
model.save('model.h5') 

C:\Users\91877\Anaconda\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [130]:
anime_ids = np.array(list(set(filtered_user.anime_id_encoded)))

In [297]:
anime_size = anime_ids.shape[0]
anime_size

7550

In [627]:
def user_anime_recommendations(user_id,anime_id,model,similarity_matrix,filtered_user,data):
    
    anime_ids = np.array(list(set(filtered_user.anime_id_encoded)))
    anime_size = anime_ids.shape[0]

    user_ids = np.array([user_id]*anime_size)
    predictions = model.predict([user_ids,anime_ids])
    
    p = predictions.flatten()
    s = similarity_matrix[anime_id]
    
    ratings = 0.5*s + 0.5*p
    top_anime_index = ratings.argsort()[-30:][::-1]
    
    watched_anime = filtered_user[filtered_user.user_id_encoded == user_id]['anime_id_encoded']
    mask = np.isin(top_anime_index, watched_anime)
    top_unwatched_anime_index = top_anime_index[~mask]
    
    recommended_animes = []
    for i in top_unwatched_anime_index:
        anime_data = data.iloc[i]
        recommended_animes.append(anime_data['title'])
        
    return recommended_animes
    
    


In [634]:
user_anime_recommendations(860,243,model,similarity_matrix,filtered_user,data)

236/236 [==============================] - 1s 3ms/step


['Hunter x Hunter (2011)',
 'Bleach: Sennen Kessen-hen',
 'One Piece',
 'Hunter x Hunter',
 'Magi: The Labyrinth of Magic',
 'Fairy Tail',
 'Magi: The Kingdom of Magic',
 'Mahoujin Guruguru',
 'Bleach: Sennen Kessen-hen - Ketsubetsu-tan',
 'Nanatsu no Taizai: Imashime no Fukkatsu',
 'Nanatsu no Taizai',
 'JoJo no Kimyou na Bouken Part 4: Diamond wa Kudakenai',
 'Dragon Quest: Dai no Daibouken (TV)',
 'Magi: Sinbad no Bouken (TV)',
 'Dragon Quest: Dai no Daibouken (2020)',
 'Fairy Tail (2014)',
 'Seirei no Moribito',
 'Sousou no Frieren',
 'Slayers Next',
 'Dungeon ni Deai wo Motomeru no wa Machigatteiru Darou ka IV: Fuka Shou - Yakusai-hen',
 'JoJo no Kimyou na Bouken Part 3: Stardust Crusaders 2nd Season',
 'One Piece Film: Strong World',
 'Tegamibachi Reverse',
 'One Piece: Episode of Merry - Mou Hitori no Nakama no Monogatari',
 'Tegamibachi',
 'Akatsuki no Yona',
 'Nanatsu no Taizai: Mokushiroku no Yonkishi',
 'Tsubasa: Shunraiki']

In [537]:
# Assume 'user_id' is 203
user_id = 9

# Create an array of the user ID repeated for each anime
user_ids = np.array([user_id]*anime_size)

# Make predictions for all animes for this user
predictions = model.predict([user_ids, anime_ids])


236/236 [==============================] - 1s 3ms/step


In [538]:
watched_anime = filtered_user[filtered_user.user_id_encoded == user_id]['anime_id_encoded']


In [539]:
p = predictions.flatten()

In [540]:
ratings = 0.5*p + 0.5*s
ratings

array([0.36171201, 0.33064339, 0.47025333, ..., 0.41330431, 0.22153572,
       0.30728343])

In [541]:
ra = ratings.argsort()[-20:][::-1]
ra

array([3800, 2997, 4375,  475, 6631, 4922, 3695, 5005, 4586, 3987, 1696,
       2078, 5759, 4775, 3557, 3532, 4604, 4420, 3485, 5796], dtype=int64)

In [542]:



# Find the elements of rec that are in xyz
mask = np.isin(ra, watched_anime)

# Invert the mask, to get the elements of rec that are NOT in xyz
inverse_mask = ~mask

# Index rec with this inverse mask to get the elements of rec that are not in xyz
rec_filtered = ra[inverse_mask]


In [543]:
rec_filtered

array([3800, 2997, 4375,  475, 6631, 4922, 3695, 5005, 4586, 3987, 1696,
       2078, 5759, 4775, 3557, 3532, 4604, 4420, 3485, 5796], dtype=int64)

In [552]:
filtered_user1 = filtered_user.copy()
filtered_user1.set_index('anime_id_encoded', inplace=True)
recommended_animes = []
for i in rec_filtered:
    anime_data = filtered_user1.iloc[i]
    recommended_animes.append(anime_data['Anime Title'])


In [553]:
recommended_animes

['Juubee Ninpuuchou',
 'Chicchana Yukitsukai Sugar',
 'NHK ni Youkoso!',
 'Maria-sama ga Miteru Specials',
 'Chaos Dragon: Sekiryuu Seneki',
 'Shugo Chara!',
 'Kami nomi zo Shiru Sekai',
 'Full Metal Panic!',
 'Jujutsu Kaisen 2nd Season',
 'Black★Rock Shooter (TV)',
 'Steamboy',
 'Tenchi Muyou! Ryououki 3rd Season',
 'Detroit Metal City',
 'Yotsunoha',
 'D.N.Angel',
 'Campione! Matsurowanu Kamigami to Kamigoroshi no Maou',
 'Kimi no Na wa.',
 'Shiki',
 'Arcana Famiglia',
 'Gall Force 3: Stardust War']

In [431]:
# Set the index of your DataFrame to be 'anime_id_encoded'
filtered_user1 = filtered_user.copy()
filtered_user1.set_index('anime_id_encoded', inplace=True)

# Reorder the rows of the DataFrame based on the order of 'anime_id_encoded' in 'z'
filtered_user1 = filtered_user1.loc[rec_filtered]

# Reset the index of the DataFrame
filtered_user1.reset_index(inplace=True)

In [373]:
a = filtered_user[filtered_user.anime_id_encoded.isin(ra)][['anime_id','anime_id_encoded']]
rec_anime = rec.anime_id.unique()
data[data.mal_id.isin(rec_anime)]['title'].sort_values(by=ratings[rec_anime_encode])

AttributeError: 'DataFrame' object has no attribute 'anime_id_encode'

In [536]:
s = similarity_matrix[3800]

In [638]:
from sklearn.metrics.pairwise import cosine_similarity

# Assume 'anime_id' is 21
anime_id = 10

# Get the embedding of the anime
anime_embedding = model.get_layer('anime_embedding').get_weights()[0]
target_anime_embedding = anime_embedding[anime_id]

# Calculate the cosine similarity between the target anime and all animes
similarities = cosine_similarity([target_anime_embedding], anime_embedding)

# Get the indices of the top 10 similar animes
top_10_indices = similarities[0].argsort()[-10:][::-1]

# Get the IDs of the top 10 similar animes
top_10_anime_ids = anime_ids[top_10_indices]

# Now 'top_10_anime_ids' contains the IDs of the top 10 animes similar to the target anime
a = filtered_user[filtered_user.anime_id_encoded.isin(top_10_anime_ids)][['anime_id']]
rec_anime = a.anime_id.unique()
data[data.mal_id.isin(rec_anime)]['title']

10                                                 Naruto
223                                              InuYasha
243                                                Bleach
546     Naruto: Takigakure no Shitou - Ore ga Eiyuu Da...
1437                   Bleach Movie 1: Memories of Nobody
1474                                   Naruto: Shippuuden
2732                  Naruto: Shippuuden Movie 2 - Kizuna
2778                  Rurouni Kenshin: Special Techniques
6090    Shingeki no Kyojin Season 2 Movie: Kakusei no ...
6790                            Bleach: Sennen Kessen-hen
Name: title, dtype: object